/usr/local/lib/python3.6/dist-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [18]:
import pandas as pd
import pandas as pd
import numpy as np
from multiprocessing import Pool
import spacy
import nltk
import swifter

data = pd.read_json("http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Beauty_5.json.gz", lines=True)
data1 = data.head(50000)
def getNounPhrase(eachValue):
  eachValue = nlp(eachValue)
  noun_phrase_pairs = []
  for chunk in eachValue.noun_chunks:
  #   print (chunk.text)
    if len(chunk.text.split(" "))>1 and chunk.text not in nltk.corpus.stopwords.words('english'):
      noun_phrase_pairs.append(chunk.text)
  return noun_phrase_pairs

global nlp
%time nlp = spacy.load("en_core_web_md") 
from pandarallel import pandarallel
pandarallel.initialize()
%time data1['NP_1'] = data1['reviewText'].parallel_apply(getNounPhrase)


CPU times: user 16 s, sys: 598 ms, total: 16.6 s
Wall time: 16.5 s
New pandarallel memory created - Size: 2000 MB
Pandarallel will run on 2 workers
CPU times: user 898 ms, sys: 170 ms, total: 1.07 s
Wall time: 37min 34s


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [31]:
!pip install pandarallel

     |████████████████████████████████| 51kB 3.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/4f/5b/bf/ff7b72bacc2a4ba33ffe5c5c6ea6453b936ad0874e3c97655c
Successfully built pandarallel
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [1]:
!pip install swifter

In [6]:
!python -m spacy download en_core_web_md


    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_md -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en_core_web_md

    You can now load the model via spacy.load('en_core_web_md')



In [2]:
import swifter


/usr/local/lib/python3.6/dist-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [0]:
num_partitions = 10 #number of partitions to split dataframe
num_cores = 2 #number of cores on your machine

In [0]:
data1 = data.head(10000)

In [15]:
len(data1)

10000

In [16]:
from pandarallel import pandarallel
pandarallel.initialize()
%time data1['NP_1'] = data1['reviewText'].parallel_apply(getNounPhrase) 

New pandarallel memory created - Size: 2000 MB
Pandarallel will run on 2 workers
CPU times: user 318 ms, sys: 97.8 ms, total: 416 ms
Wall time: 7min 18s


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [30]:
%time data1['NP_1'] = data1['reviewText'].apply(getNounPhrase)  

KeyboardInterrupt: ignored

In [29]:
 %time data1['NP_1'] = data1['reviewText'].swifter.apply(getNounPhrase)  


CPU times: user 11min 16s, sys: 4min 39s, total: 15min 56s
Wall time: 8min 16s


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [26]:
len(data1)

1000

In [0]:
swifter?

In [0]:
data = pd.read_json("http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Beauty_5.json.gz", lines=True)

In [12]:
data.head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,7806397051,"[3, 4]",1,Very oily and creamy. Not at all what I expect...,"01 30, 2014",A1YJEY40YUW4SE,Andrea,Don't waste your money,1391040000
1,7806397051,"[1, 1]",3,This palette was a decent price and I was look...,"04 18, 2014",A60XNB876KYML,Jessica H.,OK Palette!,1397779200
2,7806397051,"[0, 1]",4,The texture of this concealer pallet is fantas...,"09 6, 2013",A3G6XNM240RMWA,Karen,great quality,1378425600
3,7806397051,"[2, 2]",2,I really can't tell what exactly this thing is...,"12 8, 2013",A1PQFP6SAJ6D80,Norah,Do not work on my face,1386460800
4,7806397051,"[0, 0]",3,"It was a little smaller than I expected, but t...","10 19, 2013",A38FVHZTNQ271F,Nova Amor,It's okay.,1382140800


In [0]:
data = data.head(1000)

In [0]:
def parallelize_dataframe(df, func):
    df_split = np.array_split(df, num_partitions)
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [0]:
def getNounPhrase(eachValue):
  eachValue = nlp(eachValue)
  noun_phrase_pairs = []
  for chunk in eachValue.noun_chunks:
  #   print (chunk.text)
    if len(chunk.text.split(" "))>1 and chunk.text not in nltk.corpus.stopwords.words('english'):
      noun_phrase_pairs.append(chunk.text)
  return noun_phrase_pairs

In [16]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [12]:
# data = data.head(10000)
global nlp
%time nlp = spacy.load("en_core_web_md")


CPU times: user 15.7 s, sys: 518 ms, total: 16.2 s
Wall time: 16.2 s


In [0]:
data1 = data.head(5000)

In [0]:
%time data1['NP'] = data1.reviewText.apply(getNounPhrase)

In [0]:
num_partitions = 100 #number of partitions to split dataframe
num_cores = 2 #number of cores on your machine
def multiply_columns(data):
    data1['NP_new'] = data1['reviewText'].apply(getNounPhrase)
    return data
    
%time iris = parallelize_dataframe(data1, multiply_columns)

In [0]:
data1['reviewText']

In [0]:
%time data1['NP_new_2'] = data1['reviewText'].swifter.apply(getNounPhrase)

In [30]:
iris

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime,NP,NP_new
0,7806397051,"[3, 4]",1,Very oily and creamy. Not at all what I expect...,"01 30, 2014",A1YJEY40YUW4SE,Andrea,Don't waste your money,1391040000,[],[]
1,7806397051,"[1, 1]",3,This palette was a decent price and I was look...,"04 18, 2014",A60XNB876KYML,Jessica H.,OK Palette!,1397779200,"[This palette, a decent price, a few different...","[This palette, a decent price, a few different..."
2,7806397051,"[0, 1]",4,The texture of this concealer pallet is fantas...,"09 6, 2013",A3G6XNM240RMWA,Karen,great quality,1378425600,"[The texture, this concealer pallet, great cov...","[The texture, this concealer pallet, great cov..."
3,7806397051,"[2, 2]",2,I really can't tell what exactly this thing is...,"12 8, 2013",A1PQFP6SAJ6D80,Norah,Do not work on my face,1386460800,"[exactly this thing, a kind, oil-ish pasty flu...","[exactly this thing, a kind, oil-ish pasty flu..."
4,7806397051,"[0, 0]",3,"It was a little smaller than I expected, but t...","10 19, 2013",A38FVHZTNQ271F,Nova Amor,It's okay.,1382140800,"[a long time, great coverage, the price, your ...","[a long time, great coverage, the price, your ..."
5,7806397051,"[1, 2]",5,"I was very happy to get this palette, now I wi...","04 15, 2013",A3BTN14HIZET6Z,"S. M. Randall ""WildHorseWoman""",Very nice palette!,1365984000,"[this palette, subscription form, the makeup, ...","[this palette, subscription form, the makeup, ..."
6,7806397051,"[1, 3]",1,PLEASE DONT DO IT! this just rachett the palet...,"08 16, 2013",A1Z59RFKN0M5QL,"tasha ""luvely12b""",smh!!!,1376611200,"[the palette size, the picture, the colors, yo...","[the palette size, the picture, the colors, yo..."
7,7806397051,"[0, 1]",2,"Chalky,Not Pigmented,Wears off easily,Not a Co...","09 4, 2013",AWUO9P6PL1SY8,TreMagnifique,"Chalky, Not Pigmented, Wears off easily, Not a...",1378252800,"[Not Pigmented, Not a Coastal Scents Dupe, Too...","[Not Pigmented, Not a Coastal Scents Dupe, Too..."
8,9759091062,"[0, 0]",2,Did nothing for me. Stings when I put it on. I...,"07 13, 2014",A3LMILRM9OC3SA,NaN,"no Lightening, no Brightening,......NOTHING",1405209600,"[the whole tube, no Lightening, no Brightening...","[the whole tube, no Lightening, no Brightening..."
9,9759091062,"[0, 0]",3,I bought this product to get rid of the dark s...,"12 27, 2013",A30IP88QK3YUIO,Amina Bint Ibraheem,Its alright,1388102400,"[this product, the dark spots, my face, acne s...","[this product, the dark spots, my face, acne s..."
